# Sweeps notebook

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pprint
import os
import re
import json
import sys
import wandb
from model import *
from pytorch_lightning.loggers import WandbLogger

c:\Users\Jesli's Laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
# Define sweep config
sweep_config = {
    'method': 'random' # random, grid, or bayes
    }

In [3]:
# Define metric
metric = {
    'name': 'val_loss',
    'goal': 'minimize'   # minimize or maximize
    }
# add in sweep_config
sweep_config['metric'] = metric

In [4]:
epochs = 50
architecture = 'CNN'
dataset = 'Poems'
# Define hyperparameter space
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'embedding_dim': {
        'values': [128, 256, 512]
        },
    }
# we can indicate the distribution for continuous variables
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.0001,
        'max': 0.1
      }
    })
# we set values that we want to track but don't want to change, just indicate 1 value
parameters_dict.update({
    'epochs': {'value': epochs},
    "architecture":{'value': architecture},
    "dataset": {'value': dataset},        
    })
# add params in sweep_config
sweep_config['parameters'] = parameters_dict

In [5]:
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'architecture': {'value': 'CNN'},
                'dataset': {'value': 'Poems'},
                'embedding_dim': {'values': [128, 256, 512]},
                'epochs': {'value': 50},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0.0001},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [6]:
# Log in to weights and biases
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [7]:
sweep_id = wandb.sweep(sweep_config, project="mlo-final-project")

Create sweep with ID: 39xyia0g
Sweep URL: https://wandb.ai/msds_mlops2023_lt2/mlo-final-project/sweeps/39xyia0g


In [8]:
# define hyperparam search function
def hyperparam_search(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        # use config params
        learning_rate = config.learning_rate
        optimizer = config.optimizer
        embedding_dim = config.embedding_dim
        epochs = config.epochs

        # define model
        _model = poem_classifier_model()

        _model.load_data()
        _model.preprocess()
        _model.train(embd_dim=embedding_dim, epochs=epochs, lr=learning_rate, optimizer=optimizer)

        for i in range(epochs):
            wandb.log({"train_loss": _model.trained_model.history['loss'][i],
                       "train_acc": _model.trained_model.history['acc'][i],
                       "val_loss": _model.trained_model.history['val_loss'][i], 
                       "val_acc": _model.trained_model.history['val_acc'][i]})
        
        results = _model.test()
        wandb.log({"test_loss": results[0], "test_acc": results[1]})

wandb.agent(sweep_id, hyperparam_search, count=10) # count - num iters

wandb: Agent Starting Run: mtrt0ksy with config:
wandb: 	architecture: CNN
wandb: 	dataset: Poems
wandb: 	embedding_dim: 512
wandb: 	epochs: 50
wandb: 	learning_rate: 0.023490384701382473
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jjdsantiago3 (msds_mlops2023_lt2). Use `wandb login --relogin` to force relogin


c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:36: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('shade.When', 'shade. When')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:37: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now', 'afraid. Now')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now,', 'afraid. Now,')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Po

Epoch 1/50
21/21 [==============================] - 2s 42ms/step - loss: 1.3850 - acc: 0.2735 - val_loss: 1.3810 - val_acc: 0.3214
Epoch 2/50
21/21 [==============================] - 0s 21ms/step - loss: 1.3806 - acc: 0.2870 - val_loss: 1.3764 - val_acc: 0.3214
Epoch 3/50
21/21 [==============================] - 0s 21ms/step - loss: 1.3764 - acc: 0.2945 - val_loss: 1.3729 - val_acc: 0.3274
Epoch 4/50
21/21 [==============================] - 1s 25ms/step - loss: 1.3743 - acc: 0.2855 - val_loss: 1.3702 - val_acc: 0.3155
Epoch 5/50
21/21 [==============================] - 0s 21ms/step - loss: 1.3720 - acc: 0.2975 - val_loss: 1.3681 - val_acc: 0.3155
Epoch 6/50
21/21 [==============================] - 0s 22ms/step - loss: 1.3716 - acc: 0.2810 - val_loss: 1.3666 - val_acc: 0.2976
Epoch 7/50
21/21 [==============================] - 0s 20ms/step - loss: 1.3701 - acc: 0.2930 - val_loss: 1.3653 - val_acc: 0.2976
Epoch 8/50
21/21 [==============================] - 0s 20ms/step - loss: 1.3689 - a

test_acc,▁
test_loss,▁
train_acc,▁▃▄▃▂▄▅▆▇▂▃▃▅▅▄▅▅▇▂▄▇▇▆▆█▆█▅▄▇▄▇▅█▆▆▇▄▅█
train_loss,█▇▅▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▂▁▂▁▁
val_acc,▇▇█▇▅▅▆▆▄▃▄▃▂▂▄▄▃▄▄▃▃▂▂▂▂▂▃▃▃▃▃▃▂▂▃▃▁▃▃▃
val_loss,█▇▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,0.14667
test_loss,1.59997
train_acc,0.32138
train_loss,1.36399
val_acc,0.27976


wandb: Sweep Agent: Waiting for job.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Job received.
wandb: Agent Starting Run: atwgi5jn with config:
wandb: 	architecture: CNN
wandb: 	dataset: Poems
wandb: 	embedding_dim: 256
wandb: 	epochs: 50
wandb: 	learning_rate: 0.023417529893785864
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Agent Starting Run: kt439d3z with config:
wandb: 	architecture: CNN
wandb: 	dataset: Poems
wandb: 	embedding_dim: 128
wandb: 	epochs: 50
wandb: 	learning_rate: 0.09571373664412952
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/50


c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:36: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('shade.When', 'shade. When')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:37: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now', 'afraid. Now')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now,', 'afraid. Now,')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Po

21/21 [==============================] - 1s 15ms/step - loss: 1.4297 - acc: 0.3274 - val_loss: 1.2609 - val_acc: 0.3690
Epoch 2/50
21/21 [==============================] - 0s 8ms/step - loss: 1.0703 - acc: 0.5590 - val_loss: 1.5500 - val_acc: 0.4167
Epoch 3/50
21/21 [==============================] - 0s 8ms/step - loss: 0.9367 - acc: 0.6293 - val_loss: 1.7858 - val_acc: 0.3036
Epoch 4/50
21/21 [==============================] - 0s 9ms/step - loss: 0.8187 - acc: 0.6846 - val_loss: 1.5879 - val_acc: 0.4286
Epoch 5/50
21/21 [==============================] - 0s 9ms/step - loss: 0.7428 - acc: 0.7324 - val_loss: 2.1152 - val_acc: 0.3690
Epoch 6/50
21/21 [==============================] - 0s 8ms/step - loss: 0.7526 - acc: 0.7250 - val_loss: 2.1896 - val_acc: 0.3810
Epoch 7/50
21/21 [==============================] - 0s 8ms/step - loss: 0.6954 - acc: 0.7489 - val_loss: 2.1712 - val_acc: 0.3929
Epoch 8/50
21/21 [==============================] - 0s 8ms/step - loss: 0.5392 - acc: 0.8012 - val_l

test_acc,▁
test_loss,▁
train_acc,▁▄▅▅▆▆▇▇▇█▇▇█▇▇███▇█████████████████████
train_loss,█▆▅▄▄▃▂▂▂▂▂▂▁▂▁▁▁▂▂▁▂▂▂▁▂▂▂▂▂▁▁▁▂▁▂▂▂▂▂▂
val_acc,▅▇▂█▆▆▆▆▃▇▄▂▃▂▅▃▄▁▅▄▅▄▃▅▄▄▃▃▃▄▃▄▃▄▃▅▂▅▄▅
val_loss,▁▁▂▁▂▂▃▃▃▃▄▄▄▄▄▅▅▆▅▅▆▅▅▅▆▆▆▆▆▆▇▇▇█▇██▇▇█
test_acc,0.25333
test_loss,8.53791
train_acc,0.89537
train_loss,0.46126
val_acc,0.35714


In [9]:
wandb.finish()